## Overview ##

This is a starter notebook inspired by last year's [Logistic Regression on Tournament Seeds by Kasper P. Lauritzen](https://www.kaggle.com/kplauritzen/notebookde27b18258?scriptVersionId=804590) starter kernel. It creates a basic logistic regression model based on the seed differences between teams. 

Note that the predictions for Stage 1's sample submissions file are already based on known outcomes, and the Tourney data this model is trained on includes that data. For Stage 2, you will be predicting future outcomes based on the teams selected for the tournament on March 11.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('mode.chained_assignment', None)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV


In [ ]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
main_data_dir = '../input/mens-machine-learning-competition-2018/'
elo_data_dir = '../input/fivethirtyeight-elo-ratings/'

# Any results you write to the current directory are saved as output.

In [ ]:
print(check_output(["ls", main_data_dir]).decode("utf8"))

## Load the training data ##
I'm building off of the starter notebook by including DetailedResults for a "past 10 game" average, and season ending ELO ratings from Liam Kirwin

In [ ]:
df_seeds = pd.read_csv(main_data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(main_data_dir + 'NCAATourneyCompactResults.csv')
df_elo_ratings = pd.read_csv(elo_data_dir + 'season_elos.csv')

In [ ]:
df_elo_ratings = df_elo_ratings.rename(columns={'team_id':'WTeamID', 'season': 'Season'})
df_elo_ratings.head()

In [ ]:
df_elo_ratings[(df_elo_ratings['Season'] == 1985) & (df_elo_ratings['WTeamID'] == 1325)]

First, we'll simplify the datasets to remove the columns we won't be using and convert the seedings to the needed format (stripping the regional abbreviation in front of the seed).

In [ ]:
def seed_to_int(seed):
    #Get just the digits from the seeding. Return as int
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_seeds.head()

In [ ]:
df_tour.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.head()

## Merge seed for each team ##
Merge the Seeds and ELO ratings with their corresponding TeamIDs in the compact results dataframe.

In [ ]:
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.head()

In [ ]:
df_concat = pd.merge(left=df_concat, right=df_elo_ratings, how='left', on=['Season', 'WTeamID'])

In [ ]:
df_concat = df_concat.rename(columns={'season_elo': 'WTeamELO'})
df_elo_ratings = df_elo_ratings.rename(columns={'WTeamID': 'LTeamID'})

In [ ]:
df_concat = pd.merge(left=df_concat, right=df_elo_ratings, how='left', on=['Season', 'LTeamID'])

In [ ]:
df_concat = df_concat.rename(columns={'season_elo': 'LTeamELO'})

In [ ]:
df_concat['ELODiff'] = df_concat.WTeamELO - df_concat.LTeamELO

In [ ]:
df_concat.head()

In [ ]:
df_concat.tail()

In [ ]:
# Sanity check
df_concat[(df_concat['Season'] == 2003) & (df_concat['WTeamID'] == 1112) & (df_concat['LTeamID'] == 1436)]

Now we'll create a dataframe that summarizes wins & losses along with their corresponding seed differences. This is the meat of what we'll be creating our model on.
*First edit:* Use ELO ratings instead of seed difference for Round 1 submission

In [ ]:
df_wins = pd.DataFrame()
df_wins['ELODiff'] = df_concat['ELODiff']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['ELODiff'] = -df_concat['ELODiff']
df_losses['Result'] = 0

df_predictions = pd.concat((df_wins, df_losses))
df_predictions.head()

In [ ]:
print(df_predictions.shape)

In [ ]:
X_train = df_predictions.ELODiff.values.reshape(-1,1)
print(X_train.shape)
y_train = df_predictions.Result.values
print(y_train)
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
y_train.shape

## Train the model ##
Use a basic logistic regression to train the model. You can set different C values to see how performance changes.

In [ ]:
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=10)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

## Format season and team IDs from the SampleSubmissionStage2.csv file ##


In [ ]:
df_sample_sub = pd.read_csv(main_data_dir + 'SampleSubmissionStage2.csv')
n_test_games = len(df_sample_sub)

def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

In [ ]:
# Confirm that the df_elo_ratings still has complete (1985-2018) data
df_elo_ratings.Season.sort_values().unique()

In [ ]:
# Rename team ID column back to the generic form
df_elo_ratings = df_elo_ratings.rename(columns={'LTeamID': 'TeamID'})

In [ ]:
df_elo_ratings.tail()

## Changed function from sample notebook to grab ELO Ratings##
Create predictions using the logistic regression model we trained.

In [ ]:
X_test = np.zeros(shape=(n_test_games, 1))
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_elo = df_elo_ratings[(df_elo_ratings.TeamID == t1) & (df_elo_ratings.Season == year)].season_elo.values[0]
    t2_elo = df_elo_ratings[(df_elo_ratings.TeamID == t2) & (df_elo_ratings.Season == year)].season_elo.values[0]
    diff_elo = t1_elo - t2_elo
    X_test[ii, 0] = diff_elo

## Make Predictions ##
Create predictions using the logistic regression model we trained.

In [ ]:
preds = clf.predict_proba(X_test)[:,1]
df_sample_sub.Pred = preds
df_sample_sub.head()

Lastly, create your submission file!

In [ ]:
df_sample_sub.to_csv('dan_douthit_elo_predictions_2018.csv', index=False)